In [1]:
import numpy as np
from unbreakable.utils.data_generator import generate_households, generate_asset_damage
from unbreakable.modules.household_recovery import precompute_reconstruction_rates

## Generate households and asset damage

In [2]:
# Specify the number of households and spatial units
num_households = 300
num_spatial_units = 3
seed = 42

# Generate dummy household survey data
households = generate_households(num_households, num_spatial_units, seed)
households.to_csv("../../data/processed/household_survey/Example.csv", index=False)

# Get how much stock each household is exposed to
households["total_exposed_stock"] = (
    households["keff"] * households["household_weight"]
)

exposure_data = (
    households.groupby("spatial_unit")["total_exposed_stock"].sum().reset_index()
)


# Generate dummy disaster risk and household survey data
disaster_types = ["flood", "hurricane", "earthquake"]

for disaster in disaster_types:
    df = generate_asset_damage(disaster, exposure_data, num_spatial_units, seed=seed)
    df.to_csv(
        f"../../data/processed/asset_impacts/Example/{disaster}.csv", index=False
    )

In [3]:
exposure_data.round()

,spatial_unit,total_exposed_stock
0,region_0,30421757.0
1,region_1,37445729.0
2,region_2,28830486.0


In [4]:
df['total_exposed_stock'].iloc[:3]

0    30421757.25
1    37445728.57
2    28830486.13
Name: total_exposed_stock, dtype: float64

## Calculate recovery rates to speed up simulation

In [4]:
# NOTE These parameters must match the ones used in the config file Example.yaml
economic_params = {
    "average_productivity": 0.25,
    "consumption_utility": 1.5,
    "discount_rate": 0.04,
}
recovery_params = {"max_years": 10, "lambda_increment": 0.01}

# Generate vulnerabilities with exact 0.01 step
dwelling_vulnerabilities = np.round(np.arange(0.2, 0.91, 0.01), 2)

optimal_rates = precompute_reconstruction_rates(
    economic_params, recovery_params, dwelling_vulnerabilities
)

optimal_rates.to_csv(
        "../../data/generated/Example/optimal_reconstruction_rates.csv", index=False
    )